In [70]:
import os, errno
import numpy as np
import xarray as xr
import dask
import distributed
#from distributed import Client
#import netCDF4

In [71]:
from py3grads import Grads
from datetime import datetime, timedelta

In [72]:
ga = Grads(verbose=False)

In [73]:
def create_netcdf(mfile,nreg,var_list,base_path,f_dateanl,f_datefct,mem,fname,Vars):
    ga("open " + mfile)
    if nreg == "GL":
        ilat = -90; flat = 90
        ilon = 0  ; flon = 360
    elif nreg == "NH":
        ilat = 20 ; flat = 90
        ilon = 0  ; flon = 360
    elif nreg == "TR":
        ilat = -20; flat = 20
        ilon = 0  ; flon = 360
    elif nreg == "SH":
        ilat = -90; flat = -20
        ilon = 0  ; flon = 360
    elif nreg == "AS":
        ilat = -60; flat = 15
        ilon = 270; flon = 330

    ga(f"set lat {ilat} {flat}")
    ga(f"set lon {ilon} {flon}")
    
    v_list = []
    for var in var_list:
        nvar = Vars[var][1]
        data = ga.exp(nvar)
        menv = ga.env()
        ipts = menv.nx
        jpts = menv.ny
        lons = np.linspace(ilon,flon,ipts)
        lats = np.linspace(ilat,flat,jpts)
        da_moddata = xr.DataArray(data,                         \
                                  name="field",                 \
                                  coords={"dateanl": f_dateanl, \
                                          "datefct": f_datefct, \
                                          "lat": lats,          \
                                          "lon": lons},         \
                                  dims=["lat", "lon"])
        v_list.append(da_moddata)

    da_lstdata = xr.concat(v_list, dim="vars")

    ga("close 1")

    memdir = base_path + "/nc/" + f_dateanl + "/" + mem + "/"
    try:
        os.makedirs(memdir)
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise

#    da_lstdata.to_netcdf(memdir + fname + ".nc")
    da_lstdata.chunk().to_netcdf(memdir + fname + ".nc")
#    c = distributed.Client()
#    da_lstdata.chunk().to_netcdf(memdir + fname + ".nc", engine="netcdf4")
#    c.stop_worker()


In [74]:
c = distributed.Client()
#c = Client(processes=False)

/opt/anaconda3/lib/python3.6/site-packages/distributed/bokeh/core.py:57: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn('\n' + msg)


In [75]:
c

Client Scheduler: tcp://127.0.0.1:42327 Dashboard: http://127.0.0.1:34111/status,Cluster Workers: 4 Cores: 8 Memory: 8.31 GB


In [76]:
def main():
#    c = distributed.Client()
#    c

    datei = datetime.strptime("2018110112", "%Y%m%d%H")
    datef = datetime.strptime("2018110112", "%Y%m%d%H")

    Fincs = np.arange(00,384,24)

    ctrname = "NMC"
    Mems = ["01N", "01P", "02N", "02P", \
            "03N", "03P", "04N", "04P", \
            "05N", "05P", "06N", "06P", \
            "07N", "07P", ctrname]

    var_list = ["t", "z", "p", "q"]
    var = "t"
    Vars = {
            "z": ["z500","zgeo(lev=500)","gpm","Geopotential Height"], \
            "p": ["psml","psnm","hPa","Sea Level Pressure"],           \
            "t": ["t850","temp(lev=850)","K","Absolute Temperature"],  \
            "q": ["q925","umes(lev=925)","kg/kg","Specific Humidity"]
           }
    reg = "GL"

    resol = "TQ0126L028"

    base_path = "/home/carlos/Documents/INPE2019/ENSEMBLE/ens_tq0126l028"

    date = datei
    while (date <= datef):

        for mem in Mems:

            dateanl = date
            f_dateanl = dateanl.strftime("%Y%m%d%H")

            for finc in Fincs:

                datefct = date + timedelta(hours=int(finc))
                f_datefct = datefct.strftime("%Y%m%d%H")

                print(f_dateanl,f_datefct,mem,finc)

                yyyy = str(f_dateanl[0:4])
                mm = str(f_dateanl[4:6])
                dd = str(f_dateanl[6:8])
                hh = str(f_dateanl[8:10])
                
                modpath = base_path + "/grb/" + mem + "/" + yyyy + "/" + mm + "/" + dd + "/" + hh + "/"

                if finc == 00:
                    fname = "GPOS" + mem + f_dateanl + f_dateanl + "P.icn." + resol
                    modfnam = fname + ".ctl"
                else:
                    fname = "GPOS" + mem + f_dateanl + f_datefct + "P.fct." + resol
                    modfnam = fname + ".ctl"

                modfile = modpath + modfnam

                create_netcdf(modfile,reg,var_list,base_path,f_dateanl,f_datefct,mem,fname,Vars)

        date = date + timedelta(hours=24)

#    c.stop_worker()

In [77]:
%%time
if __name__ == "__main__":
    main()

2018110112 2018110112 01N 0
2018110112 2018110212 01N 24
2018110112 2018110312 01N 48
2018110112 2018110412 01N 72
2018110112 2018110512 01N 96
2018110112 2018110612 01N 120
2018110112 2018110712 01N 144
2018110112 2018110812 01N 168
2018110112 2018110912 01N 192
2018110112 2018111012 01N 216
2018110112 2018111112 01N 240
2018110112 2018111212 01N 264
2018110112 2018111312 01N 288
2018110112 2018111412 01N 312
2018110112 2018111512 01N 336
2018110112 2018111612 01N 360
2018110112 2018110112 01P 0
2018110112 2018110212 01P 24
2018110112 2018110312 01P 48
2018110112 2018110412 01P 72
2018110112 2018110512 01P 96
2018110112 2018110612 01P 120
2018110112 2018110712 01P 144
2018110112 2018110812 01P 168
2018110112 2018110912 01P 192
2018110112 2018111012 01P 216
2018110112 2018111112 01P 240
2018110112 2018111212 01P 264
2018110112 2018111312 01P 288
2018110112 2018111412 01P 312
2018110112 2018111512 01P 336
2018110112 2018111612 01P 360
2018110112 2018110112 02N 0
2018110112 2018110212 02